## Importing Libraries

In [1]:
import os
import shutil
import glob
from pathlib import Path
from dotenv import load_dotenv


import torch
from roboflow import Roboflow

In [2]:
load_dotenv()

True

In [3]:
print(torch.__version__)

print(torch.backends.mps.is_available())

print(torch.backends.mps.is_built())

2.2.1
True
True


## Setting Roboflow API Key

In [4]:
roboflow_api_key = os.getenv("roboflow_api_key")
rf = Roboflow(api_key=roboflow_api_key)

## Setting the Dataset Paths

In [44]:
logo_dataset_path = "./datasets/brand-logo"
groceries_dataset_path = "./datasets/groceries"
shelves_dataset_path = "./datasets/shelves"
complete_dataset_path = "./datasets/complete_dataset"
product_dataset_path = "./datasets/products"

## Downloading the Annotated datasets with Bounding Boxes from Roboflow

In [6]:
shelf_project = rf.workspace("shelfdetect-yzkro").project("shelves-ugxt3")
shelf_dataset = shelf_project.version(3).download("yolov8", location=shelves_dataset_path, overwrite=True)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ./datasets/shelves in yolov8:: 100%|██████████| 2472/2472 [00:00<00:00, 4740.91it/s]


In [7]:
grocery_project = rf.workspace("grocery-barcodes").project("sku110k-1000-2000")
grocery_dataset = grocery_project.version(3).download("yolov8", location=groceries_dataset_path, overwrite=True)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ./datasets/groceries in yolov8:: 100%|██████████| 5562/5562 [00:01<00:00, 3564.98it/s]


In [8]:
logo_project = rf.workspace("philip-hawkins-ocdim").project("openlogo-zgtgi")
logo_dataset = logo_project.version(4).download("yolov8", location=logo_dataset_path, overwrite=True)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ./datasets/brand-logo in yolov8:: 100%|██████████| 19246/19246 [00:03<00:00, 5137.56it/s]


In [46]:
product_project = rf.workspace("grocery-barcodes").project("sku110k-1000-2000")
product_dataset = product_project.version(3).download("yolov8", location=product_dataset_path, overwrite=True)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ./datasets/products in yolov8:: 100%|██████████| 5562/5562 [00:01<00:00, 3918.05it/s]


In [9]:
if os.path.exists(complete_dataset_path):
    shutil.rmtree(complete_dataset_path)

In [10]:
for split in ["train", "test", "valid"]:
    os.makedirs(f"{complete_dataset_path}/{split}/images", exist_ok=True)
    os.makedirs(f"{complete_dataset_path}/{split}/labels", exist_ok=True)

## Preprocess Shelves, Groceries and Brand Logo Datasets

In [11]:
def preprocess_brand_logo_dataset(brand_dataset_path, brand_class=174, brand_name="kelloggs"):
    for file_path in glob.glob(brand_dataset_path + "/*/labels/*.txt"):
        with open(file_path, "r+") as fh:
            annotations = fh.readlines()
            fh.seek(0)
            for annotation in annotations:
                annotation_list = annotation.split(" ")
                if int(annotation_list[0]) == brand_class:
                    annotation_list[0] = "1"
                else:
                    annotation_list[0] = "0"
                new_annotation = " ".join(annotation_list)
                fh.write(new_annotation)

In [12]:
preprocess_brand_logo_dataset(logo_dataset_path)

In [13]:
def preprocess_shelves_dataset(shelves_dataset_path, shelf_class=0):
    for file_path in glob.glob(shelves_dataset_path + "/*/labels/*.txt"):
        with open(file_path, "r+") as fh:
            annotations = fh.readlines()
            fh.seek(0)
            for annotation in annotations:
                annotation_list = annotation.split(" ")
                if int(annotation_list[0]) == shelf_class:
                    annotation_list[0] = "2"
                else:
                    raise Exception("Unexpected shelf class label encountered")
                new_annotation = " ".join(annotation_list)
                fh.write(new_annotation)

In [14]:
preprocess_shelves_dataset(shelves_dataset_path)

In [15]:
def preprocess_groceries_dataset(groceries_dataset_path, gorcery_label=0):
    for file_path in glob.glob(groceries_dataset_path + "/*/labels/*.txt"):
        with open(file_path, "r+") as fh:
            annotations = fh.readlines()
            fh.seek(0)
            for annotation in annotations:
                annotation_list = annotation.split(" ")
                if int(annotation_list[0]) == gorcery_label:
                    annotation_list[0] = "3"
                else:
                    raise Exception("Unexpected grocery class label encountered")
                new_annotation = " ".join(annotation_list)
                fh.write(new_annotation)

In [16]:
preprocess_groceries_dataset(groceries_dataset_path)

## Merging Datasets

In [17]:
def merge_datasets(dataset_paths):
    for dataset_path in dataset_paths:
        for split in ["train", "test", "valid"]:
            for file_path in glob.glob(dataset_path + f"/{split}/images/*.jpg"):
                shutil.copy(file_path, f"{complete_dataset_path}/{split}/images")
            for file_path in glob.glob(dataset_path + f"/{split}/labels/*.txt"):
                shutil.copy(file_path, f"{complete_dataset_path}/{split}/labels")

In [18]:
merge_datasets([groceries_dataset_path, shelves_dataset_path, logo_dataset_path])

In [19]:
with open(complete_dataset_path + "/data.yaml", "w") as fh:
    data = """
names:
- other
- kelloggs
- shelf
- product
nc: 4
path: /Users/priyanshutuli/Desktop/Planogram_Optimization/datasets/complete_dataset
test: ./test/images
train: ./train/images
val: ./valid/images
\n
    """
    fh.write(data)

## Setting Comet ML API Key

In [20]:
import comet_ml

In [21]:
comet_ml_api_key = os.getenv("comet_ml_api_key")

In [22]:
comet_ml.init(api_key=comet_ml_api_key)

COMET INFO: Valid Comet API Key saved in /Users/priyanshutuli/.comet.config (set COMET_CONFIG to change where it is saved).


## YoloV8 Object Detection

In [23]:
from ultralytics import YOLO
from ultralytics.engine import trainer

In [24]:
model_xlarge = YOLO('yolov8x.pt', task="detect")

In [25]:
model_xlarge.names.values()

dict_values(['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'])

In [26]:
len(model_xlarge.names)

80

## Setting Model Configs

In [27]:
inference_images_path = "./inference_images"

In [28]:
seed = 0
device = "cpu"
image_size = 640
batch_size = 16

## Zero-Shot Learning

In [29]:
# zero_shot_results = model_xlarge.predict(inference_images_path, conf=0.5, imgsz=640, iou=0.7, save=True, device=device, seed=seed, verbose=False)

## Setting Data Yaml Path

In [47]:
data_yaml_path = "/Users/priyanshutuli/Desktop/Planogram_Optimization/datasets/products/data.yaml"

## Retraining all the Layers of the model

In [31]:
model_nano = YOLO("yolov8n.pt", task="detect")

In [32]:
# model_nano.train(data=data_yaml_path, epochs=20, imgsz=image_size, seed=seed, device=device, save=True, patience=10, batch=batch_size, verbose=True,
#                       plots=True, cos_lr=True, lr0=1e-3, lrf=1e-8, optimizer='AdamW', deterministic=True, save_period=1, amp=True, val=True, augment=True)

In [33]:
# retraining_results = model_nano.predict(inference_images_path, conf=0.7, imgsz=image_size, iou=0.75, save=True, device=device, seed=seed)

## Getting Class Distribution and Split Distribution

In [34]:
def get_label_distribution(annotation_dir):
    class_distribution = {}
    split_distribution = {"train": 0, "test": 0, "valid": 0}

    # Iterate through each annotation file
    for split in split_distribution.keys():
        for file_path in glob.glob(annotation_dir + f"/{split}/labels/*.txt"):
            with open(file_path, 'r') as fh:
                lines = fh.readlines()
                for line in lines:
                    class_id = line.split()[0]
                    label = f'class_{class_id}'

                    # Update label distribution
                    if label in class_distribution:
                        class_distribution[label] += 1
                    else:
                        class_distribution[label] = 1
        split_distribution[split] = len(os.listdir(annotation_dir + f"/{split}/labels/"))
    return class_distribution, split_distribution

In [35]:
get_label_distribution(complete_dataset_path)

({'class_3': 407094,
  'class_0': 17937,
  'class_2': 10960,
  'class_1': 167,
  'class_228': 2,
  'class_262': 2,
  'class_237': 2,
  'class_312': 1,
  'class_242': 2,
  'class_143': 9,
  'class_313': 1,
  'class_266': 1,
  'class_240': 1,
  'class_202': 1,
  'class_302': 1,
  'class_97': 1},
 {'train': 10312, 'test': 1250, 'valid': 2060})

## Removing classes other than the required classes present in the 3 different Splits

In [36]:
def remove_other_classes(annotation_dir):
    splits = ["train", "test", "valid"]
    req_classes = [0, 1, 2, 3]
    for split in splits:
        for file_path in glob.glob(annotation_dir + f"/{split}/labels/*.txt"):
            image_file_removed_flag = 0
            with open(file_path, 'r') as fh:
                lines = fh.readlines()
                for line in lines:
                    class_id = int(line.split()[0])
                    if class_id not in req_classes:
                        image_filepath = file_path.replace("labels", "images")
                        image_filepath = image_filepath.replace(".txt", ".jpg")
                        if os.path.exists(image_filepath):
                            print(f"Removing image file: {image_filepath} because it has labels outside the required class labels")
                            os.remove(image_filepath)
                            image_file_removed_flag = 1
                            break
            if image_file_removed_flag:
                if not os.path.exists(file_path):
                    raise Exception(f"Unexpected situation encountered. Corrseponding label file not present for {image_filepath} removed")
                print(f"Remvoing the corresponding label file as well: {file_path}")
                os.remove(file_path)    

In [37]:
remove_other_classes(complete_dataset_path)

Removing image file: ./datasets/complete_dataset/train/images/niveaimg000264_jpg.rf.1ad12d53bc9b5a78166f670d75c3a972.jpg because it has labels outside the required class labels
Remvoing the corresponding label file as well: ./datasets/complete_dataset/train/labels/niveaimg000264_jpg.rf.1ad12d53bc9b5a78166f670d75c3a972.txt
Removing image file: ./datasets/complete_dataset/train/images/reebokimg000017_jpg.rf.5c3202d50cbe1fbea3c7ca91a8b33773.jpg because it has labels outside the required class labels
Remvoing the corresponding label file as well: ./datasets/complete_dataset/train/labels/reebokimg000017_jpg.rf.5c3202d50cbe1fbea3c7ca91a8b33773.txt
Removing image file: ./datasets/complete_dataset/train/images/Pampersimg000209_jpg.rf.b5b83d2340df4dbcdf6034b50882d437.jpg because it has labels outside the required class labels
Remvoing the corresponding label file as well: ./datasets/complete_dataset/train/labels/Pampersimg000209_jpg.rf.b5b83d2340df4dbcdf6034b50882d437.txt
Removing image file: .

In [38]:
get_label_distribution(complete_dataset_path)

({'class_3': 407094, 'class_0': 17253, 'class_2': 10960, 'class_1': 167},
 {'train': 10300, 'test': 1250, 'valid': 2055})

## Freezing the Backbone of the YoloV8 Model

In [39]:
def freeze_layer(trainer):
    model = trainer.model
    num_freeze = 10
    print(f"Freezing {num_freeze} layers")
    freeze = [f'model.{x}.' for x in range(num_freeze)]  # layers to freeze 
    for k, v in model.named_parameters(): 
        v.requires_grad = True  # train all layers 
        if any(x in k for x in freeze): 
            print(f'freezing {k}') 
            v.requires_grad = False 
    print(f"{num_freeze} layers are freezed.")

In [48]:
model_nano_froozen_layers = YOLO(model="yolov8n.pt", task="detect")

In [50]:
checkpoint_path = "./runs/detect/product_model"

In [51]:
model_nano_froozen_layers.train(data=data_yaml_path, epochs=50, imgsz=image_size, seed=seed, device=device, save=True, patience=10, batch=batch_size, verbose=True,
                      plots=True, cos_lr=True, lr0=1e-3, lrf=1e-6, optimizer='AdamW', deterministic=True, save_period=1, amp=True, val=True, augment=True, dropout=0.3, freeze=10, save_dir=checkpoint_path)

New https://pypi.org/project/ultralytics/8.1.27 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.11 torch-2.2.1 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/priyanshutuli/Desktop/Planogram_Optimization/datasets/products/data.yaml, epochs=50, patience=10, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=cpu, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, overlap_mask=True, mask_ratio=4, dropout=0.3, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visua

## Resuming Training from Last Checkpoint

In [95]:
# last_checkpoint_path = "/Users/priyanshutuli/Desktop/Planogram_Optimization/runs/detect/train56/weights/last.pt"
# model_resume_training_froozen_layers = YOLO(last_checkpoint_path, task="detect")
# model_resume_training_froozen_layers.train(resume=True)

In [63]:
# model_resume_training_froozen_layers = YOLO("/Users/priyanshutuli/Desktop/Planogram_Optimization/runs/detect/train5/weights/last.pt", task="detect")

## Predicting on the inference images

In [43]:
model_nano_froozen_layers.predict(inference_images_path, conf=0.5, imgsz=image_size, iou=0.9, save=True, device=device, seed=seed)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/999 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/0205BB3A-F171-4F92-A552-46B64F835D30_IMG_3467.jpeg: 640x480 1 other, 127.3ms
image 2/999 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/0F488364-219C-4A21-94B2-0B6A884420DD_IMG_0041.jpeg: 640x480 8 products, 70.9ms
image 3/999 /Users/priyanshutuli/Desktop/Planogram_Optimization/inference_images/12B22766-4862-470A-BC05-DAB41EFB537D_IMG_0986.jpeg: 640x4

KeyboardInterrupt: 